In [13]:
import numpy as np
from scipy import misc
import os
import time

In [14]:
# Householder
def make_householder(a):
    u = a + np.copysign(np.linalg.norm(a), a[0])
    v = a / u[0]
    v[0] = 1
    H = np.eye(a.shape[0])
    beta = 2 / (np.dot(v, v.transpose()))
    vtv = np.dot(np.matrix(v).transpose(), np.matrix(v))
    H -= np.dot(beta, vtv)
    return H

def qrDecomposition(A):
    m, n = A.shape
    Q = np.eye(m)
    for i in range(n - (m == n)):
        H = np.eye(m)
        H[i:, i:] = make_householder(A[i:, i])
        Q = np.dot(Q, H)
        A = np.dot(H, A)
    return Q, A

def qr(toCompute, maxIter = 100):
    A = []
    Q = np.eye(toCompute.shape[0])
    A.append(None)
    A.append(toCompute)
    for k in range(maxIter):
        A[0] = A[1]
        q, R = qrDecomposition(A[0])
        A[1] = np.dot(R, q)
        Q = Q.dot(q)
    return np.diagonal(A[1]), Q

In [15]:
def concatMatrix(mtxLst):
    mtx = np.array(())
    flatLst = []
    for m in mtxLst:
        flatLst.append(m.flatten())
    mtx = np.vstack(flatLst)
    return mtx.transpose()

def extractEigenvecOnVal(eigval, eigvec, threshold = 1):
    delIdx = np.where(eigval < threshold)[0]
    return np.delete(eigvec, delIdx, axis=1)

def reconstructVector(M, eigvec):
    eigvecT = eigvec.transpose()
    szeNewM = (np.shape(eigvec)[1], np.shape(M)[0])
    newmatrix = np.empty(szeNewM)
    for idx, vec in enumerate(eigvecT):
        newvec = np.dot(M, vec.transpose())
        newmatrix[idx] = newvec
    return newmatrix.transpose()

def computeCostMulti(X, y, theta):
    H = np.dot(X, theta)
    diff = H.transpose() - y
    diff = np.power(diff, 2)
    sdiff = np.sum(diff, axis=1)
    cost = sdiff / (2. * (np.shape(y)[0]))
    return cost.item(0)

def gradDescent(X, y, theta, alpha, numIter = None):
    if numIter is None:
        return gradDescentConvergence(X, y, theta, alpha)
    return gradDescentIteration(X, y, theta, alpha, numIter)

def gradDescentIteration(X, y, theta, alpha, numIter):
    m = np.shape(y)[0]
    for i in range(numIter):
        H = np.dot(X, theta)
        diff = H.transpose() - y
        sigma = np.dot(X.transpose(), diff.transpose()) / m
        theta = theta - alpha * sigma
    print "Last Iteration Cost: ", computeCostMulti(X, y, theta)
    return theta

def gradDescentConvergence(X, y, theta, alpha):
    m = np.shape(y)[0]
    i = 0
    cost = computeCostMulti(X, y, theta)
    costp = cost + 1
    diff = costp - cost
    while (diff > 1e-100):
        H = np.dot(X, theta)
        diff = H.transpose() - y
        sigma = np.dot(X.transpose(), diff.transpose()) / m
        theta = theta - alpha * sigma
        costp = cost
        cost = computeCostMulti(X, y, theta)
        diff = costp - cost
        i = i + 1
    print "Convergence Cost (", i + 1,  "iteration ): ", computeCostMulti(X, y, theta)
    return theta

In [16]:
learRate = 1e-11
maxIteration = 300
img_extension = ".pgm"
trainPath = "../faceset/sample/train/"
validPath = "../faceset/sample/valid/"

In [17]:
usleep = lambda x: time.sleep(x/1000000.0)

def loadmatrixs(path):
    matrixs = []
    sze = len(os.listdir(path))
    for i, filename in enumerate(os.listdir(path)):
        if not filename.endswith(img_extension):
            continue
        img = misc.imread(path + filename)
        matrixs.append((filename, img))
        usleep(250)
    return matrixs

In [18]:
def train(mtxLst):
    thetas = list()
    M = concatMatrix(mtxLst)
    Mmean = M.mean(axis=1)
    M -= Mmean[:, np.newaxis]
    Mtld = np.dot(M.transpose(), M)
    n = np.shape(Mtld)[1]
    #eigenval, eigenvec = qr(Mtld, 400)
    eigenval, eigenvec = np.linalg.eig(Mtld)
    eigenvec = extractEigenvecOnVal(eigenval, eigenvec, 1)
    eigenvec = reconstructVector(M, eigenvec)
    print "eigenvec shape: ", np.shape(eigenvec)
    for i in range(n):
        img = M.transpose()[i]
        theta = np.ones(np.shape(eigenvec)[1])
        theta = gradDescent(eigenvec, np.matrix(img), np.matrix(theta).transpose(), learRate, maxIteration)
        print "Descent terminated: ", i, " / ", n - 1  ######
        thetas.append(theta)
    return (Mmean, eigenvec, thetas)

In [19]:
def submit(mtx, mean, eigenvec):
    mtxflat = mtx.flatten()
    mtxflat = np.vstack(list(mtxflat))
    mtxflat -= mean[:, np.newaxis]
    mtxflat = mtxflat.transpose()[0]
    theta = np.ones(np.shape(eigenvec)[1])
    theta = gradDescent(eigenvec, np.matrix(mtxflat), np.matrix(theta).transpose(), learRate, maxIteration)
    return theta

def compare(thetaSubmit, thetas):
    minIdx = 0
    minVal = np.absolute(np.sum(thetas[0] - thetaSubmit))
    for idx, theta in enumerate(thetas):
        val = np.absolute(np.sum(np.absolute(theta - thetaSubmit)))
        if val < minVal:
            minIdx = idx
            minVal = val
    return minIdx

def compareAvgGap(thetaSubmit, thetas):
    minIdx = 0
    minVal = (np.absolute(thetas[0] - thetaSubmit)).mean()
    for idx, theta in enumerate(thetas):
        val = (np.absolute(theta - thetaSubmit)).mean()
        if val < minVal:
            minIdx = idx
            minVal = val
    return minIdx

In [20]:
trainD = loadmatrixs(trainPath)
validD = loadmatrixs(validPath)

In [21]:
ftrainD, mtrainD = zip(*trainD)

In [22]:
mean, eigenvec, thetas = train(mtrainD)
count = 0

eigenvec shape:  (10304, 36)
Last Iteration Cost:  760387406.921
Descent terminated:  0  /  71
Last Iteration Cost:  837783246.123
Descent terminated:  1  /  71
Last Iteration Cost:  781212673.066
Descent terminated:  2  /  71
Last Iteration Cost:  708679080.248
Descent terminated:  3  /  71
Last Iteration Cost:  766684879.835
Descent terminated:  4  /  71
Last Iteration Cost:  753194125.107
Descent terminated:  5  /  71
Last Iteration Cost:  770338979.758
Descent terminated:  6  /  71
Last Iteration Cost:  726070636.568
Descent terminated:  7  /  71
Last Iteration Cost:  821827296.558
Descent terminated:  8  /  71
Last Iteration Cost:  816981034.661
Descent terminated:  9  /  71
Last Iteration Cost:  800618041.514
Descent terminated:  10  /  71
Last Iteration Cost:  813400052.595
Descent terminated:  11  /  71
Last Iteration Cost:  849993738.169
Descent terminated:  12  /  71
Last Iteration Cost:  869093757.78
Descent terminated:  13  /  71
Last Iteration Cost:  843223670.624
Descent 

In [23]:
print "Mean: ", mean
print "eigenvec: \n", eigenvec

Mean:  [ 96.88888889  97.19444444  97.19444444 ...,  69.55555556  68.52777778
  68.90277778]
eigenvec: 
[[  717.75543729   -13.63753755   -75.11343491 ...,    87.03227508
    -42.13666284    95.50391127]
 [  712.02391005   -12.85908678   -80.59735887 ...,    90.98844297
    -32.57341363    99.22794992]
 [  711.70314966   -11.648162     -69.88430745 ...,    98.57303777
    -29.08482165    94.70800891]
 ..., 
 [ 1139.8864403    -87.52383332   -42.57667544 ...,    76.71106459
    -75.55834238   196.68810889]
 [ 1169.70202924  -134.05159394   -93.58776596 ...,    85.28502621
    -50.61688637   156.10907307]
 [ 1142.04912069   -89.26493667   -40.97930376 ...,    89.68394148
    -64.9836286    198.36542548]]


In [26]:
for idx, data in enumerate(validD):
    success = False
    filename, mtx = data
    thetaSubmit = submit(mtx, mean, eigenvec)
    matchIdx = compareAvgGap(thetaSubmit, thetas)
    if filename[:filename.rfind("_")] == ftrainD[matchIdx][:ftrainD[matchIdx].rfind("_")]:
        success = True
        count = count + 1
    if success is True:
        print filename, " --> ", ftrainD[matchIdx], "[X]"
    else:
        print filename, " --> ", ftrainD[matchIdx], "[ ]"

Last Iteration Cost:  812776546.343
s1_0001.pgm  -->  s1_0004.pgm [X]
Last Iteration Cost:  780695337.625
s1_0002.pgm  -->  s1_0009.pgm [X]
Last Iteration Cost:  812870774.773
s2_0001.pgm  -->  s2_0004.pgm [X]
Last Iteration Cost:  845568993.761
s2_0002.pgm  -->  s2_0008.pgm [X]
Last Iteration Cost:  823057006.294
s3_0001.pgm  -->  s3_0004.pgm [X]
Last Iteration Cost:  809922519.276
s3_0002.pgm  -->  s3_0010.pgm [X]
Last Iteration Cost:  818346092.62
s4_0001.pgm  -->  s4_0004.pgm [X]
Last Iteration Cost:  773140436.63
s4_0002.pgm  -->  s4_0006.pgm [X]
Last Iteration Cost:  830363649.366
s5_0001.pgm  -->  s5_0005.pgm [X]
Last Iteration Cost:  814126588.046
s5_0002.pgm  -->  s5_0010.pgm [X]
Last Iteration Cost:  808028412.626
s6_0001.pgm  -->  s6_0005.pgm [X]
Last Iteration Cost:  779350461.56
s6_0002.pgm  -->  s6_0005.pgm [X]
Last Iteration Cost:  835108190.163
s7_0001.pgm  -->  s7_0004.pgm [X]
Last Iteration Cost:  832697502.113
s7_0002.pgm  -->  s7_0005.pgm [X]
Last Iteration Cost:  8

In [27]:
print count, " / ", idx + 1, "===>", count / (idx + 1.) * 100, "%"

18  /  18 ===> 100.0 %
